In [ ]:
import torch
import walnut
import numpy as np

In [ ]:
batches = 10
sequence = 8
in_channels = 3
hidden_channels = 5

X = walnut.randn((batches, sequence, in_channels))
W_in = walnut.randn((in_channels, hidden_channels))
B_in = walnut.randn((hidden_channels,))
W_hidden = walnut.randn((hidden_channels, hidden_channels))
B_hidden = walnut.randn((hidden_channels,))

t_x = torch.nn.Parameter(torch.from_numpy(X.data).float(), requires_grad=True)
t_w_in = torch.nn.Parameter(torch.from_numpy(W_in.T).float(), requires_grad=True)
t_b_in = torch.nn.Parameter(torch.from_numpy(B_in.data).float(), requires_grad=True)
t_w_hidden = torch.nn.Parameter(torch.from_numpy(W_hidden.T).float(), requires_grad=True)
t_b_hidden = torch.nn.Parameter(torch.from_numpy(B_hidden.data).float(), requires_grad=True)

### Forward

In [ ]:
import walnut.nn as nn

rnn = nn.models.RNN(in_channels, hidden_channels)
rnn.training_mode()
rnn.input.w = W_in
rnn.input.b = B_in
rnn.rec_layers[0].hidden.w = W_hidden
rnn.rec_layers[0].hidden.b = B_hidden

out = rnn(X)
out[0]

In [ ]:
rnn_t = torch.nn.RNN(in_channels, hidden_channels, batch_first = True)
rnn_t.weight_ih_l0 = t_w_in
rnn_t.bias_ih_l0 = t_b_in
rnn_t.weight_hh_l0 = t_w_hidden
rnn_t.bias_hh_l0 = t_b_hidden

t_out = rnn_t(t_x)[0]
t_out[0]

### Backward

In [ ]:
dy = walnut.randn(out.shape).data
t_dy = torch.nn.Parameter(torch.from_numpy(dy).float())

In [ ]:
rnn.reset_grads()
x_grad = rnn.backward(dy)

In [ ]:
t_out.backward(t_dy)

X

Problem: cannot backward multiple times or different inputs, because backward is defined during call and therefore data of the last forward pass is used for x, y, etc.

In [ ]:
x_grad[0]

In [ ]:
t_x.grad[0]

W Hidden

In [ ]:
rnn.rec_layers[0].hidden.w.grad

In [ ]:
rnn_t.weight_hh_l0.grad.T

B Hidden

In [ ]:
rnn.rec_layers[0].hidden.b.grad

In [ ]:
rnn_t.bias_hh_l0.grad

W Input

In [ ]:
rnn.input.w.grad

In [ ]:
rnn_t.weight_ih_l0.grad.T

B Input

In [ ]:
rnn.input.b.grad

In [ ]:
rnn_t.bias_ih_l0.grad